In [16]:
import os
import pickle
import numpy as np
from PIL import Image
from collections import OrderedDict

In [10]:
model_list = []

In [11]:
for file in os.listdir("./rotate"):
    if file.startswith("model_name_"):
        print(os.path.join("./rotate", file))
        with open(os.path.join("./rotate", file), "rb") as f:
            model_list_partial = pickle.load(f)
        for model in model_list_partial:
            model_list.append(model.split("/")[-2])
    #break

./rotate\model_name_0
./rotate\model_name_1000
./rotate\model_name_2000
./rotate\model_name_3000
./rotate\model_name_5000
./rotate\model_name_6000
./rotate\model_name_7000
./rotate\model_name_8000
./rotate\model_name_9000


In [13]:
syn_loss = np.zeros(shape=len(model_list))

9000

In [15]:
for idx, model in enumerate(model_list):
    print(model)
    break
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            syn_img_path = "./rotate/rotate/" + str(idx * 1000 + az * 10 + el) + ".0.jpg"
            realimg = np.array(Image.open(real_img_path))
            synimg = np.array(Image.open(syn_img_path))
            syn_loss[idx] += np.mean(np.abs(realimg - synimg))
    syn_loss[idx] /= 54
print(np.mean(syn_loss) / 256. * 3)

7b2dcc7c4c879e9116f7ad0d1093bd39


In [ ]:
model_path = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\02958343\\02958343"
refine_loss = np.zeros(shape=len(model_list))
for idx, model in enmerate(model_list):
    print(model)
    break
    os.system("blender --background --python 3dloc_2dcoor.py -- --views 18 " + os.path.join(model_path, model, "model.obj"))
    with open("./vertex2pixel_" + model, "rb") as f:
        vertex2pixel = pickle.load(f)
    with open("./pixel2vertex_" + model, "rb") as f:
        pixel2vertex = pickle.dump(f)
    vertex2color = OrderedDict()
    for az in range(18):
        for el in range(3):
            syn_img_path = "./rotate/rotate/" + str(idx * 1000 + az * 10 + el) + ".0.jpg"
            for i in range(128):
                for j in range(128):
                    if pixel2vertex[az, el, i, j, 0] != 1234567.0:
                        if not repr(pixel2vertex[az, el, i, j, 1:]) in vertex2color:
                            vertex2color[repr(pixel2vertex[az, el, i, j, 1:])] = []
                        vertex2color[repr(pixel2vertex[az, el, i, j, 1:])].append(syn_img_path[i, j])
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            syn_img_path = "./rotate/rotate/" + str(idx * 1000 + az * 10 + el) + ".0.jpg"
            realimg = np.array(Image.open(real_img_path))
            synimg = np.array(Image.open(syn_img_path))
            
            synimg_refined = synimg.copy()
            for i in range(128):
                for j in range(128):
                    if pixel2vertex[az, el, i, j, 0] != 1234567.0:
                        synimg_refined[i][j] = np.mean(vertex2color[repr(pixel2vertex[az, el, i, j, 1:])], axis=0)
            refine_loss[idx] += np.mean(np.abs(realimg - synimg_refined))
    refine_loss[idx] /= 54
    print("synloss: " + syn_loss[idx] + ", refinedloss: " + refine_loss[idx])
    
print(np.mean(refine_loss) / 256. * 3)